# 3. MapReduce

For this exercise we are going to use MapReduce in local mode, i.e. we won't be running anything on the cluster!
 
## 3.1. Use the commands `head`, `cat`, `uniq`, `wc`, `sort`, `find`, `xargs`, `awk` to evaluate the NASA log file:

* Data File:  <https://github.com/scalable-infrastructure/exercise-2018/blob/master/data/nasa/NASA_access_log_Jul95.gz>
* Which page was called the most?
* What was the most frequent return code?
* How many errors occurred? What is the percentage of errors?


In [2]:
DATA_DIR="/home/hpc/ug201/di57hah/admin/exercise-2019/data"
!pwd
!gzip -k -d -f  {DATA_DIR}/nasa/NASA_access_log_Jul95.gz

/home/hpc/ug201/di57hah/admin/exercise-2019/03_MapReduce


In [3]:
!ls ../data/nasa

NASA_access_log_Jul95  NASA_access_log_Jul95.gz


In [31]:
!head ../data/nasa/NASA_access_log_Jul95

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085
burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0
199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0
205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985
d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 

In [38]:
!cat ../data/nasa/NASA_access_log_Jul95 | awk  '{print $(NF-3)}' | sort | uniq -c | sort -r |head

awk: cmd. line:1: (FILENAME=- FNR=1891715) fatal: attempt to access field -2
 111086 /images/NASA-logosmall.gif
  89529 /images/KSC-logosmall.gif
  60299 /images/MOSAIC-logosmall.gif
  59844 /images/USA-logosmall.gif
  59324 /images/WORLD-logosmall.gif
  58615 /images/ksclogo-medium.gif
  40841 /images/launch-logo.gif
  40250 /shuttle/countdown/
  40067 /ksc.html
  33555 /images/ksclogosmall.gif
sort: write failed: 'standard output': Broken pipe
sort: write error


In [4]:
%%time
!cat ../data/nasa/NASA_access_log_Jul95 | awk  '{print $(NF-1)}'| sort | uniq -c

1701534 200
  46573 302
 132627 304
      5 400
     54 403
  10845 404
     62 500
     14 501
      1 alyssa.p
CPU times: user 44 ms, sys: 4.43 ms, total: 48.4 ms
Wall time: 2.25 s


Compute Percentages

In [5]:
data = !cat ../data/nasa/NASA_access_log_Jul95 | awk  '{print $(NF-1)}'| sort | uniq -c

In [6]:
import pandas as pd
df=pd.DataFrame.from_records([i.split() for i in data], columns=["Count", "HTTP RC"])
df["Count"]=pd.to_numeric(df["Count"], errors='coerce')
df["Counts_pct"]=(df["Count"]/df["Count"].sum()*100) 
df

,Count,HTTP RC,Counts_pct
0,1701534,200,89.946636
1,46573,302,2.461946
2,132627,304,7.010940
3,5,400,0.000264
4,54,403,0.002855
5,10845,404,0.573289
6,62,500,0.003277
7,14,501,0.000740
8,1,alyssa.p,0.000053


## 3.2 Implement a Python version of this Unix Shell script using this script as template! Run the Python script inside an Hadoop Streaming job.

Template: <https://github.com/scalable-infrastructure/scalable-infrastructure.github.io/blob/master/src/map_reduce.py>

In [15]:
import os
TMP_DIR=os.path.join(os.getcwd(), "tmp")
os.environ["HADOOP_HOME"]="/naslx/projects/ug201/di57hah/hadoop-2.8.5"
os.environ["JAVA_HOME"]="/lrz/sys/compilers/java/jdk1.8.0_112"
os.environ["JAVA_OPTS"]="-Djava.io.tmpdir="+TMP_DIR
os.environ["HADOOP_OPTS"]="-Djava.io.tmpdir="+TMP_DIR
os.environ["PATH"]=os.path.join(os.environ["HADOOP_HOME"], "bin") + ":"+os.environ["PATH"]

In [49]:
!head -n 100 ../data/nasa/NASA_access_log_Jul95 | python map_reduce_solution.py map| sort | python map_reduce_solution.py reduce

Traceback (most recent call last):
  File "map_reduce_solution.py", line 33, in <module>
Traceback (most recent call last):
  File "map_reduce_solution.py", line 33, in <module>
    import __builtin__
ModuleNotFoundError: No module named '__builtin__'
    import __builtin__
ModuleNotFoundError: No module named '__builtin__'


In [16]:
!hadoop jar ${HADOOP_HOME}/share/hadoop/tools/lib/hadoop-streaming-2.8.5.jar -info

Java HotSpot(TM) 64-Bit Server VM warning: Insufficient space for shared memory file:
   15159
Try using the -Djava.io.tmpdir= option to select an alternate temp location.

Usage: $HADOOP_PREFIX/bin/hadoop jar hadoop-streaming.jar [options]
Options:
  -input          <path> DFS input file(s) for the Map step.
  -output         <path> DFS output directory for the Reduce step.
  -mapper         <cmd|JavaClassName> Optional. Command to be run as mapper.
  -combiner       <cmd|JavaClassName> Optional. Command to be run as combiner.
  -reducer        <cmd|JavaClassName> Optional. Command to be run as reducer.
  -file           <file> Optional. File/dir to be shipped in the Job jar file.
                  Deprecated. Use generic option "-files" instead.
  -inputformat    <TextInputFormat(default)|SequenceFileAsTextInputFormat|JavaClassName>
                  Optional. The input format class.
  -outputformat   <TextOutputFormat(default)|JavaClassName>
                  Optional. The output fo

In [44]:
%%time
!rm -rf nasa-out
!hadoop  jar ${HADOOP_HOME}/share/hadoop/tools/lib/hadoop-streaming-2.8.5.jar \
             -input `pwd`/../data/nasa/NASA_access_log_Jul95 -output nasa-out \
             -mapper "map_reduce_solution.py map" -reducer "map_reduce_solution.py reduce"  \
             -file `pwd`/map_reduce_solution.py  

Java HotSpot(TM) 64-Bit Server VM warning: Insufficient space for shared memory file:
   21835
Try using the -Djava.io.tmpdir= option to select an alternate temp location.

19/04/15 14:01:13 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/home/hpc/ug201/di57hah/admin/exercise-2019/03_MapReduce/map_reduce_solution.py] [] /home/hpc/ug201/di57hah/admin/exercise-2019/03_MapReduce/tmp/streamjob4966106445513744564.jar tmpDir=null
19/04/15 14:01:14 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
19/04/15 14:01:14 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
19/04/15 14:01:14 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
19/04/15 14:01:14 INFO mapred.FileInputFormat: Total input files to process : 1
19/04/15 14:01:14 INFO mapreduce.JobSubmitter: number of splits:7
19/04/15 14:01

---

## 3.3 Run the program Terasort on 1 GB of data - each record that TeraGen generates is 100 Bytes in size:

    hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar teragen <number_of_records> <output_directory>

    hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar terasort <input_directory> <output_directory>

Measure the runtime for each step!

In [1]:
%%time
!hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.8.5.jar teragen 10000000 teragen-1GB

/usr/bin/sh: hadoop: command not found
CPU times: user 11.2 ms, sys: 8.6 ms, total: 19.8 ms
Wall time: 185 ms


In [9]:
%%time
!hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.8.5.jar teragen 10000000 terasort-1GB

18/04/03 15:03:21 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
18/04/03 15:03:21 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
18/04/03 15:03:22 INFO terasort.TeraSort: Generating 10000000 using 1
18/04/03 15:03:22 INFO mapreduce.JobSubmitter: number of splits:1
18/04/03 15:03:22 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local722040421_0001
18/04/03 15:03:23 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
18/04/03 15:03:23 INFO mapreduce.Job: Running job: job_local722040421_0001
18/04/03 15:03:23 INFO mapred.LocalJobRunner: OutputCommitter set in config null
18/04/03 15:03:23 INFO output.FileOutputCommitter: File Output Committer Algorithm version is 1
18/04/03 15:03:23 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter
18/04/03 15:03:23 INFO mapred.LocalJobRunner: Waiting for map tasks
18/04/03 15:03:23 I

In [19]:
%%time
!hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar teravalidate terasort-1GB teravalidate-1GB

18/03/11 22:30:55 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
18/03/11 22:30:55 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
18/03/11 22:30:56 INFO input.FileInputFormat: Total input paths to process : 1
Spent 282ms computing base-splits.
Spent 15ms computing TeraScheduler splits.
18/03/11 22:30:56 INFO mapreduce.JobSubmitter: number of splits:1
18/03/11 22:30:56 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1518461612_0001
18/03/11 22:30:57 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
18/03/11 22:30:57 INFO mapreduce.Job: Running job: job_local1518461612_0001
18/03/11 22:30:57 INFO mapred.LocalJobRunner: OutputCommitter set in config null
18/03/11 22:30:57 INFO output.FileOutputCommitter: File Output Committer Algorithm version is 1
18/03/11 22:30:57 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapreduce.lib.output.FileOutputCommitte